In [3]:
import numpy as np
import random
import pandas as pd

In [4]:
n=4
A = np.array([[random.random() for i in range(n)] for j in range(n)])
A0 = np.copy(A)
for i in range(n):
    A[i,i]=sum(abs(A[i,:]))
print(A)

[[0.80123451 0.13780526 0.31216401 0.07847971]
 [0.94466666 2.44758246 0.51302322 0.18136096]
 [0.58632541 0.92070157 1.90314929 0.21000292]
 [0.41030029 0.43437848 0.05861599 1.73501433]]


# Метод Якоби
### Составим матрицу В и вектор С
$$B = E - D^{-1}A \ \ \ \ \ C=D^{-1}b$$

In [5]:
diagonal=np.diag(A)
invDiagonal = diagonal
D=np.diag(diagonal)
invD=np.zeros([n,n])
for i in range (n):
    invD[i,i] = 1/D[i,i]
print(f'D=\n{D}')
print(f'invD=\n{invD}')

D=
[[0.80123451 0.         0.         0.        ]
 [0.         2.44758246 0.         0.        ]
 [0.         0.         1.90314929 0.        ]
 [0.         0.         0.         1.73501433]]
invD=
[[1.24807406 0.         0.         0.        ]
 [0.         0.40856642 0.         0.        ]
 [0.         0.         0.52544485 0.        ]
 [0.         0.         0.         0.57636412]]


In [6]:
b=np.array([random.random() for i in range(n)])
B=np.eye(n)-invD@A
C=invD@b
xJ=C
B

array([[ 1.11022302e-16, -1.71991168e-01, -3.89603805e-01,
        -9.79484908e-02],
       [-3.85959075e-01,  1.11022302e-16, -2.09604059e-01,
        -7.40979982e-02],
       [-3.08081669e-01, -4.83777903e-01,  1.11022302e-16,
        -1.10344954e-01],
       [-2.36482365e-01, -2.50360171e-01, -3.37841528e-02,
         0.00000000e+00]])

### Выполним априорную оценку

In [7]:
Eps=1e-12
Bnorm=np.linalg.norm(B)
cntPrior=0
Bk=1
while Bk*Bnorm/(1-Bnorm)*np.linalg.norm(C)>Eps:
    cntPrior+=1
    Bk*=Bnorm
print(f'Априорная оценка = {cntPrior}')

Априорная оценка = 372


### Выполним апостериорную оценку

In [6]:
cntPost=0

In [8]:
x_new=xJ
Bnorm=np.linalg.norm(B)
Bk=1
while True:
    xJ=x_new
    cntPost=cntPost+1
    x_new=B@xJ+C
    if np.linalg.norm(x_new-xJ)<Eps :
        break
xJ=x_new
print(f'Апостериорная оценка = {cntPost}')
print(np.linalg.solve(A,b)-xJ)
print(A@xJ-b)


Апостериорная оценка = 86
[-1.98174810e-13 -1.65478742e-13 -1.71179043e-13 -1.71002101e-13]
[2.63233879e-13 6.45483667e-13 5.61509172e-13 7.32303107e-13]


# Метод Зейделя

In [9]:
print(A)

[[0.76862048 0.15978974 0.29266931 0.20115272]
 [0.55294273 2.25694131 0.01823037 0.93092365]
 [0.090969   0.64032439 1.89838031 0.65844047]
 [0.92057823 0.19756782 0.54529779 2.47806121]]


In [23]:

D = np.diag(np.diag(A))
L = np.tril(A, k=-1)             # нижняя треугольная
U = np.triu(A, k=1)              #верхняя треугольная


DplusL = D + L
DplusLinv = np.linalg.inv(DplusL)
B = -DplusLinv @ U  

Eps=1e-12
Bnorm=np.linalg.norm(B)
cntPrior=0
Bk=1
while Bk*Bnorm/(1-Bnorm*np.linalg.norm(C))>Eps: #log
    cntPrior+=1
    Bk*=Bnorm
print(f'Априорная оценка = {cntPrior}')

xS=C
cntZ=1
cnt=10000
while cnt!=0:
    x_new = np.copy(xS)
    for i in range(n):
        # Сумма элементов строки i, умноженных на новые x[j] (j < i)
        sumLeft = np.dot(A[i, :i], x_new[:i])
        # Сумма элементов строки i, умноженных на старые x[j] (j > i)
        sumRight = np.dot(A[i, i+1:], xS[i+1:])
        x_new[i] = (b[i] - sumLeft - sumRight) / A[i, i]
        cntZ+=1
    
    if Bnorm/(1-Bnormnp.linalg.norm(xS-x_new) < Eps:
        break
    xS = x_new
    cnt-=1
print(cntZ)
print(A@x_new-b)

Априорная оценка = 79
85
[9.32587341e-15 2.56905608e-13 1.83394966e-13 0.00000000e+00]


# Проверим методы на матрицах без диагонального преобладания
## Та же матрица, но без ухищрений с диагональю:

In [25]:
A0=A0@A0.T
print(A0)

[[1.2413445  1.618875   0.81371815 1.33131449]
 [1.618875   2.56490661 1.350477   1.97070438]
 [0.81371815 1.350477   0.85338575 0.93507022]
 [1.33131449 1.97070438 0.93507022 1.60596034]]


## Метод якоби

In [26]:
diagonal0=np.diag(A0)
invDiagonal0 = diagonal0
D0=np.diag(diagonal0)
invD0=np.zeros([n,n])
for i in range (n):
    invD0[i,i] = 1/D0[i,i]
print(f'D0=\n{D0}')
print(f'invD0=\n{invD0}')

D0=
[[1.2413445  0.         0.         0.        ]
 [0.         2.56490661 0.         0.        ]
 [0.         0.         0.85338575 0.        ]
 [0.         0.         0.         1.60596034]]
invD0=
[[0.80557814 0.         0.         0.        ]
 [0.         0.38987774 0.         0.        ]
 [0.         0.         1.17180302 0.        ]
 [0.         0.         0.         0.62268038]]


In [27]:
B0=np.eye(n)-invD0@A0
C0=invD0@b
xJ0=C0

In [28]:
cnt=cntPost
while cnt>0:
    cnt-=1
    xJ0=B0@xJ0+C0

print(A0@xJ0-b)

[1.80473243e+33 2.70013899e+33 1.43515798e+33 2.09493013e+33]


### Не сходится
# Метод Зейделя?

In [35]:

xS0=np.zeros(n)
cnt=cntPost+100000
while cnt!=0:
    x_new = np.copy(xS0)
    for i in range(n):
        # Сумма элементов строки i, умноженных на новые x[j] (j < i)
        sumLeft = np.dot(A0[i, :i], x_new[:i])
        # Сумма элементов строки i, умноженных на старые x[j] (j > i)
        sumRight = np.dot(A0[i, i+1:], xS0[i+1:])
        x_new[i] = (b[i] - sumLeft - sumRight) / A0[i, i]
    xS0 = x_new
    cnt-=1
print(A0@xS0-b)

[ 1.44328993e-15 -9.40358902e-14 -1.31006317e-14  4.60742555e-14]
